In [2]:
!pip install python-barcode pillow ipywidgets

  Using cached python_barcode-0.15.1-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.0 MB/s eta 0:00:00


In [15]:
import csv
import os
from barcode import Code128
from barcode.writer import ImageWriter
from PIL import Image
from ipywidgets import widgets, VBox, HBox
from IPython.display import display, clear_output

CSV_FILE = "inventory.csv"
BARCODE_DIR = "barcodes"

if not os.path.exists(BARCODE_DIR):
    os.makedirs(BARCODE_DIR)

def generate_barcode(product_id):
    barcode = Code128(str(product_id), writer=ImageWriter())
    filepath = os.path.join(BARCODE_DIR, f"{product_id}.png")
    barcode.save(filepath)
    return filepath

def save_to_csv(data):
    headers = ["ProductID", "ProductName", "Category", "Price", "Quantity"]
    file_exists = os.path.isfile(CSV_FILE)
    with open(CSV_FILE, mode="a", newline="") as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(headers)
        writer.writerow(data)

def load_inventory():
    if not os.path.isfile(CSV_FILE):
        return []
    with open(CSV_FILE, mode="r") as file:
        reader = csv.DictReader(file)
        reader.fieldnames = ["ProductID", "ProductName", "Category", "Price", "Quantity"]
        return list(reader)

def add_product(change=None):
    clear_output(wait=True)
    product_id = product_id_input.value
    product_name = product_name_input.value
    category = category_input.value
    price = price_input.value
    quantity = quantity_input.value

    if not product_id or not product_name or not category or not price or not quantity:
        display(widgets.HTML("<h3 style='color: red;'>All fields are required!</h3>"))
    elif any(product["ProductID"] == product_id for product in load_inventory()):
        display(widgets.HTML("<h3 style='color: red;'>Product ID already exists!</h3>"))
    else:
        save_to_csv([product_id, product_name, category, price, quantity])
        barcode_path = generate_barcode(product_id)

        display(widgets.HTML("<h3 style='color: green;'>Product added successfully!</h3>"))
        display(widgets.HTML(f"<b>Barcode for {product_name}:</b>"))
        display(Image.open(barcode_path))
        display(widgets.HTML(f"<b>CSV saved as '{CSV_FILE}'.</b>"))

def view_inventory(change=None):
    clear_output(wait=True)
    inventory = load_inventory()
    if not inventory:
        display(widgets.HTML("<h3>No products in inventory.</h3>"))
    else:
        display(widgets.HTML("<h3>Inventory:</h3>"))
        for product in inventory:
            display(widgets.HTML(
                f"ID: {product['ProductID']}, Name: {product['ProductName']}, "
                f"Category: {product['Category']}, Price: {product['Price']}, Quantity: {product['Quantity']}"
            ))

def scan_barcode(change=None):
    clear_output(wait=True)
    barcode_id = scan_input.value
    inventory = load_inventory()
    product = next((p for p in inventory if p['ProductID'] == barcode_id), None)
    if product:
        display(widgets.HTML("<h3>Product Details:</h3>"))
        display(widgets.HTML(
            f"ID: {product['ProductID']}, Name: {product['ProductName']}, "
            f"Category: {product['Category']}, Price: {product['Price']}, Quantity: {product['Quantity']}"
        ))
    else:
        display(widgets.HTML("<h3 style='color: red;'>Product not found!</h3>"))

# Widgets
product_id_input = widgets.Text(placeholder="Product ID:", description="ID:")
product_name_input = widgets.Text(placeholder="Product Name:", description="Name:")
category_input = widgets.Dropdown(
    options=["Electronics", "Clothing", "Groceries", "Books", "Furniture", "Others"],
    value="Electronics",
    description="Category:"
)
price_input = widgets.FloatText(description="Price:")
quantity_input = widgets.IntText(description="Quantity:")
add_button = widgets.Button(description="Add Product")
view_button = widgets.Button(description="View Inventory")
scan_input = widgets.Text(description="Scan Barcode:")
scan_button = widgets.Button(description="Scan")

add_button.on_click(add_product)
view_button.on_click(view_inventory)
scan_button.on_click(scan_barcode)

display(widgets.HTML("<h2>Inventory and Barcode Manager</h2>"))
display(VBox([HBox([product_id_input, product_name_input]),
              HBox([category_input, price_input, quantity_input]),
              add_button,
              view_button,
              HBox([scan_input, scan_button])]))


HTML(value="<h3 style='color: green;'>Product added successfully!</h3>")

HTML(value='<b>Barcode for Developer Sweatshirt:</b>')

FileNotFoundError: [Errno 2] No such file or directory: '/content/barcodes/Sweatshirts.png'